In [141]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from dsto_gan import DSTO_GAN

In [142]:
import pickle

# Abre o arquivo .pkl usando pickle
with open('Titanic.pkl', 'rb') as f:
    data = pickle.load(f)

# Verifica a estrutura dos dados carregados
print("Tipo de dados carregados:", type(data))
print("Conteúdo dos dados:", data)

# Se for uma lista, acesse os elementos por índice
if isinstance(data, list):
    X_treino = data[0]
    X_teste = data[1]
    y_treino = data[2]
    y_teste = data[3]
    feature_names = data[4] if len(data) > 4 else None  # Verifica se feature_names existe
else:
    raise TypeError("O arquivo .pkl não contém uma lista. Verifique a estrutura dos dados.")

# Verifica os dados carregados
print("X_treino:", X_treino)
print("X_teste:", X_teste)
print("y_treino:", y_treino)
print("y_teste:", y_teste)
print("Feature Names:", feature_names)

Tipo de dados carregados: <class 'list'>
Conteúdo dos dados: [     Pclass  Sex   Age  SibSp  Parch      Fare
331       1    1  45.5      0      0   28.5000
733       2    1  23.0      0      0   13.0000
382       3    1  32.0      0      0    7.9250
704       3    1  26.0      1      0    7.8542
813       3    0   6.0      4      2   31.2750
..      ...  ...   ...    ...    ...       ...
106       3    0  21.0      0      0    7.6500
270       1    1  30.0      0      0   31.0000
860       3    1  41.0      2      0   14.1083
435       1    0  14.0      1      2  120.0000
102       1    1  21.0      0      1   77.2875

[712 rows x 6 columns],      Pclass  Sex   Age  SibSp  Parch     Fare
709       3    1  30.0      1      1  15.2458
439       2    1  31.0      0      0  10.5000
840       3    1  20.0      0      0   7.9250
720       2    0   6.0      0      1  33.0000
39        3    0  14.0      1      0  11.2417
..      ...  ...   ...    ...    ...      ...
433       3    1  17.0     

In [143]:
print("Distribuição original:", y_treino.value_counts())

Distribuição original: Survived
0    444
1    268
Name: count, dtype: int64


In [144]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_treino, y_treino)

print("\nDistribuição após SMOTE:", y_train_smote.value_counts())

# Treinar modelo
model = RandomForestClassifier(random_state=42)
model.fit(X_train_smote, y_train_smote)
y_pred = model.predict(X_teste)


Distribuição após SMOTE: Survived
0    444
1    444
Name: count, dtype: int64


In [145]:
print("\nSMOTE - Relatório de Classificação:")
print(classification_report(y_teste, y_pred))


SMOTE - Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       105
           1       0.76      0.76      0.76        74

    accuracy                           0.80       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



In [146]:
from imblearn.under_sampling import TomekLinks

tomek = TomekLinks()
X_train_tomek, y_train_tomek = tomek.fit_resample(X_treino, y_treino)

print("\nDistribuição após Tomek Links:", y_train_tomek.value_counts())

# Treinar modelo
model = RandomForestClassifier(random_state=42)
model.fit(X_train_tomek, y_train_tomek)
y_pred = model.predict(X_teste)


Distribuição após Tomek Links: Survived
0    403
1    268
Name: count, dtype: int64


In [147]:
print("\nTomek Links - Relatório de Classificação:")
print(classification_report(y_teste, y_pred))


Tomek Links - Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       105
           1       0.77      0.77      0.77        74

    accuracy                           0.81       179
   macro avg       0.80      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179



In [148]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_treino, y_treino)

print("\nDistribuição após Random Under-sampling:", y_train_rus.value_counts())

# Treinar modelo
model = RandomForestClassifier(random_state=42)
model.fit(X_train_rus, y_train_rus)
y_pred = model.predict(X_teste)


Distribuição após Random Under-sampling: Survived
0    268
1    268
Name: count, dtype: int64


In [149]:
print("\nRandom Under-sampling - Relatório de Classificação:")
print(classification_report(y_teste, y_pred))


Random Under-sampling - Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.85      0.79      0.82       105
           1       0.73      0.80      0.76        74

    accuracy                           0.79       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.79      0.79       179



In [150]:
# 2. Métodos de balanceamento tradicionais (para comparação)
methods = {
    "SMOTE": SMOTE(random_state=42),
    "TomekLinks": TomekLinks(),
    "RandomUnderSampler": RandomUnderSampler(random_state=42)
}

for name, method in methods.items():
    X_res, y_res = method.fit_resample(X_treino, y_treino)
    model = RandomForestClassifier(random_state=42)
    model.fit(X_res, y_res)
    y_pred = model.predict(X_teste)
    
    print(f"\n{name} - Classification Report:")
    print(classification_report(y_teste, y_pred))

# 3. Solução alternativa para DSTO-GAN (caso o pacote não funcione)
# Implementaremos uma abordagem similar com GAN básica usando TensorFlow/Keras

try:
    from dsto_gan import DSTO_GAN
    # Tentativa com o pacote original
    dstogan = DSTO_GAN(epochs=50, batch_size=32)  # Reduzi epochs para demo
    X_res_dsto, y_res_dsto = dstogan.fit_resample(X_treino.values, y_treino.values)
    
    model = RandomForestClassifier(random_state=42)
    model.fit(X_res_dsto, y_res_dsto)
    y_pred = model.predict(X_teste)
    
    print("\nDSTO-GAN (original) - Classification Report:")
    print(classification_report(y_teste, y_pred))

except Exception as e:
    print(f"\nErro ao usar DSTO-GAN: {str(e)}")
    print("Implementando alternativa com GAN básica...")
    
    # Implementação alternativa simples
    from tensorflow.keras.layers import Dense, Input
    from tensorflow.keras.models import Model, Sequential
    from tensorflow.keras.optimizers import Adam
    
    # Configuração da GAN simples
    def build_generator():
        model = Sequential([
            Dense(32, input_dim=6, activation='relu'),
            Dense(16, activation='relu'),
            Dense(6, activation='tanh')
        ])
        return model
    
    # Apenas para demonstração - não é o DSTO-GAN real
    minority = X_treino[y_treino == 1].values
    generator = build_generator()
    
    # Gerar amostras sintéticas (número igual à classe majoritária)
    noise = np.random.normal(0, 1, size=(len(X_treino[y_treino == 0]) - len(minority), 6))
    synthetic = generator.predict(noise)
    
    # Combinar dados
    X_combined = np.vstack([X_treino.values, synthetic])
    y_combined = np.hstack([y_treino.values, np.ones(len(synthetic))])
    
    # Treinar modelo
    model = RandomForestClassifier(random_state=42)
    model.fit(X_combined, y_combined)
    y_pred = model.predict(X_teste)


SMOTE - Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       105
           1       0.76      0.76      0.76        74

    accuracy                           0.80       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179


TomekLinks - Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       105
           1       0.77      0.77      0.77        74

    accuracy                           0.81       179
   macro avg       0.80      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179


RandomUnderSampler - Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.79      0.82       105
           1       0.73      0.80      0.76        74

    accuracy                           0.79       179
   macro avg  

c:\Users\llays\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [151]:
print("\nDSTO-GAN - Relatório de Classificação:")
print(classification_report(y_teste, y_pred))


DSTO-GAN - Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179

